<a href="https://colab.research.google.com/github/vanditagoyal1997/depression-analysis/blob/sankhya/Audio_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.15.2
from keras.models import Sequential
import keras
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D,Dropout
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import tensorflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import wavfile as wav
from scipy import signal
from scipy.fftpack import fft,fftfreq
import os
from keras.utils import to_categorical
from tensorflow.python.keras.layers import Dense, Conv2D, Flatten,MaxPooling2D,Dropout
from tensorflow.python.keras import Sequential
from sklearn.model_selection import train_test_split
from PIL import Image
from keras.regularizers import l2
from keras.regularizers import l1
from keras.models import model_from_json
import h5py

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15.2`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
print(tensorflow.__version__)

1.15.2


# generating spectogram images from audio data

In [ ]:
a=os.listdir()

for i in a:
    w=wave.open(i,'rb')
    frames = w.readframes(-1)
    sound_info = pylab.fromstring(frames, 'Int16')
    frame_rate = w.getframerate()
    w.close()
    pylab.figure(num=None, figsize=(513, 125))
    pylab.subplot(111)
    pylab.title('spectrogram of the audio file')
    pylab.specgram(sound_info, Fs=frame_rate)
    pylab.savefig('E:/final year project/Spectrogram/'+i[0:-3]+'spec.png')

# cnn model for the spectogram images

first we willl write a script to change the directory to the current folder and then we will read all the images

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
os.listdir()
os.chdir('gdrive')
os.chdir('My Drive/final year project')


now we will write a script to read all the spectogram images 

the cell below is only for train split and here we take all the images which are in trail split csv
this is later to be exculded when all the data has been acquired

In [ ]:
df_1=pd.read_csv('train_split_Depression_AVEC2017.csv')
df_11=df_1.rename(columns={'PHQ8_Binary':'PHQ_Binary'})
df_2=pd.read_csv('full_test_split.csv')
df_3=pd.read_csv('full_test_split.csv')
df_4=pd.concat([df_11,df_2,df_3]).drop_duplicates().reset_index(drop=True)


In [ ]:
df_3.shape,df_2.shape,df_1.shape,df_4.shape
df_11.columns,df_2.columns,df_3.columns,df_4.columns
df_4.PHQ_Binary.tolist()

In [ ]:
df=pd.read_csv('train_split_Depression_AVEC2017.csv')
id=df_4['Participant_ID'].tolist()
id=np.array(id)
score=df['PHQ8_Binary'].tolist()
score=np.array(score)
f=os.listdir('spectogram_resize')
temp=[]

for i in f:
  if len(id)==0:
    break
  elif int(i[0:3]) in id:
    temp.append(i)
len(temp) # to check all the values have been read or not

154

In [ ]:
# arr=[]
# for i in temp1:
#     t=load_img('spectogram_test/'+i)
#     x=img_to_array(t)
# #     x = x.reshape((1,) + x.shape)
#     arr.append(x)

In [ ]:
arr=[]
for i in temp:
    t=load_img('spectogram_resize/'+i)
    x=img_to_array(t)
#     x = x.reshape((1,) + x.shape)
    arr.append(x)

In [ ]:
X=np.array(arr)
X.shape

(154, 512, 512, 3)

In [ ]:
y=df_4['PHQ_Binary'].tolist()
Y=to_categorical(y,num_classes=2)
num_classes=2

In [ ]:
num_classes=2

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)
y_train[0]


array([1., 0.], dtype=float32)

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=3,strides=(1,1) , activation='relu', input_shape=(512,512,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))
model.add(Conv2D(64,  kernel_size=5,strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(128,  kernel_size=7,strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,optimizer=tensorflow.keras.optimizers.Adadelta(),metrics=['accuracy',keras.metrics.Precision(), keras.metrics.Recall(),f1 ])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


the next model is the one used in the project


In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=3,strides=(1,1) , activation='relu', input_shape=(512,512,3),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Conv2D(32,  kernel_size=3,strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3),strides=2))
model.add(Conv2D(64,  kernel_size=5,strides=(1,1), activation='relu'))
model.add(Conv2D(64,  kernel_size=5,strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3),strides=2))
model.add(Conv2D(128,  kernel_size=5,strides=(1,1), activation='relu'))
model.add(Conv2D(128,  kernel_size=5,strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3),strides=2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,optimizer=tensorflow.keras.optimizers.Adadelta(),metrics=['accuracy',keras.metrics.Precision(), keras.metrics.Recall()])

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 510, 510, 32)      896       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 508, 508, 32)      9248      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 253, 253, 32)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 249, 249, 64)      51264     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 245, 245, 64)      102464    
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 122, 122, 64)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 118, 118, 128)    

In [ ]:
json_file = open('audio.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("audio.h5")
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# load_model.evaluate(X_test,y_test)
score = loaded_model.evaluate(X_test,y_test, verbose=0)
print(score)
# loaded_model.predict(X_test)

In [ ]:
model.fit(X_train,y_train,validation_split=0.1 ,epochs=10)

In [ ]:
model.evaluate(X_test,y_test)

16/16 [==============================] - 1s 48ms/sample - loss: 0.7175 - acc: 0.5625 - precision_2: 0.6132 - recall_2: 0.6132


[0.7175301909446716, 0.5625, 0.6131805, 0.6131805]

In [ ]:
# imports for storing weights 
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
import tensorflow as tf

# saving the weights

model_json = model.to_json()
with open("audio.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("audio.h5")

# retrieving the weights

json_file = open('audio.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("audio.h5")
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# load_model.evaluate(X_test,y_test)
score = loaded_model.evaluate(X_test,y_test, verbose=0)
print(score)
# loaded_model.predict(X_test)